In [12]:
import torch
import torch.nn as nn
from torchvision.models.vgg import vgg16
from torchvision import transforms, datasets, models
from torchvision.transforms import Compose,ToTensor,Resize,RandomHorizontalFlip,RandomCrop,Normalize
from torch.utils.data.dataloader import DataLoader
import torch
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
import os
import shutil
import koreanize_matplotlib
from math import ceil
import random
import matplotlib.pyplot as plt
from PIL import Image
from collections import defaultdict

In [3]:
model = vgg16(pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(25088, 4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096, 12)
)

model


c:\Deep\deep\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Deep\deep\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
model.load_state_dict(torch.load('vgg_gamemodel.pth'))

<All keys matched successfully>

In [7]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])

In [8]:
# 이미지 데이터 폴더 경로
train_image_folder = 'C:\\Deep\\Game_model\\gamedataset\\train'
test_image_folder = 'C:\\Deep\\Game_model\\gamedataset\\test'

# 데이터셋 로드
train_dataset = datasets.ImageFolder(root=train_image_folder, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_image_folder, transform=transform_test)

# DataLoader 생성
train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [9]:
import torch
from sklearn.metrics import precision_recall_fscore_support,confusion_matrix
 
model.eval()
correct = 0
total = 0

all_preds = []
all_labels = []

with torch.no_grad():
    for img, label in test_dataloader:
        img, label = img.to(device), label.to(device)
        pred = model(img)
        result = pred.max(1)[1]
        
        # 결과와 라벨을 리스트에 추가
        all_preds.extend(result.cpu().numpy())
        all_labels.extend(label.cpu().numpy())
        
        correct += result.eq(label).sum().item()
        total += label.size(0)

accuracy = correct / total
print(f'accuracy : {accuracy}')

# precision, recall, f1-score 계산
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

accuracy : 0.9887992831541219
Precision: 0.9888170513551151
Recall: 0.9887992831541219
F1 Score: 0.9887986969443744


In [15]:
#외부데이터 별 정확도
# 이미지 폴더 경로
image_folder = 'C:\\Deep\\Game_model\\val_img'

# train_dataset.classes 정의 (여기서는 예시로 클래스 이름을 정의)
train_dataset = type('', (), {})()
train_dataset.classes = ['카운트스트라이크', '카트라이더', '크레이지아케이드', '사이퍼즈','던전앤파이터','엘소드','FC온라인','마비노기','마비노기영웅전','메이플스토리','서든어택','바람의나라']

correct_predictions = 0
total_predictions = 0

class_correct = defaultdict(int)
class_total = defaultdict(int)

# 이미지 폴더 내의 모든 파일을 순회
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # jpg나 png 파일만 처리
        image_path = os.path.join(image_folder, filename)
        
        # 이미지 열기
        image = Image.open(image_path)
        
        # 이미지 변환 적용
        image = transform_test(image)
        
        # 차원 추가 (배치 차원 추가)
        image = image.unsqueeze(0)
        
        # 이미지 텐서를 장치로 이동
        image = image.to(device)
        
        # 모델 예측
        with torch.no_grad():
            pred = model(image)
            pred = torch.max(pred, 1)[1]
            
        actual_label_name = filename.split(' ')[0]
        if actual_label_name in train_dataset.classes:
            actual_label = train_dataset.classes.index(actual_label_name)
        else:
            print(f"Unknown class in filename: {filename}")
            continue
        
        if pred.item() == actual_label:
            correct_predictions += 1
            class_correct[actual_label_name] += 1
        class_total[actual_label_name] += 1
        total_predictions += 1
        
        # 예측 결과 출력
        print(f"실제 이미지 {filename}: 예측 결과 = {train_dataset.classes[pred[0]]}")
        
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
error_rate = 1 - accuracy

# 결과 출력
print(f"Accuracy: {accuracy:.4f}, Error Rate: {error_rate:.4f}")

for class_name in train_dataset.classes:
    class_accuracy = class_correct[class_name] / class_total[class_name] if class_total[class_name] > 0 else 0
    print(f"Class: {class_name}, Accuracy: {class_accuracy:.4f}, Total: {class_total[class_name]}")


실제 이미지 FC온라인 (1).jpg: 예측 결과 = 서든어택
실제 이미지 FC온라인 (10).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (11).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (12).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (13).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (14).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (15).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (2).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (3).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (4).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (5).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (6).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (6).png: 예측 결과 = FC온라인
실제 이미지 FC온라인 (7).jpg: 예측 결과 = 서든어택
실제 이미지 FC온라인 (8).jpg: 예측 결과 = FC온라인
실제 이미지 FC온라인 (9).jpg: 예측 결과 = 서든어택
실제 이미지 던전앤파이터 (1).jpg: 예측 결과 = 마비노기
실제 이미지 던전앤파이터 (10).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (11).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (12).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (13).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (14).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (15).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (16).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (17).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (18).jpg: 예측 결과 = 던전앤파이터
실제 이미지 던전앤파이터 (19).jpg: 예측 결과 = 던전

In [16]:
#테스트 데이터 별 정확도
correct_predictions = 0
total_predictions = 0

# 각 클래스별 맞춘 예측과 전체 예측 수를 저장할 딕셔너리 초기화
class_correct = defaultdict(int)
class_total = defaultdict(int)

# 테스트 데이터로 예측 수행
for images, labels in test_dataloader:
    images, labels = images.to(device), labels.to(device)
    
    # 모델 예측
    with torch.no_grad():
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
    
    # 맞춘 경우와 틀린 경우를 계산
    for label, pred in zip(labels, preds):
        if pred == label:
            correct_predictions += 1
            class_correct[test_dataset.classes[label]] += 1
        class_total[test_dataset.classes[label]] += 1
        total_predictions += 1

# 전체 이미지에 대한 맞춘 비율과 틀린 비율 계산
accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
error_rate = 1 - accuracy

# 결과 출력
print(f"Overall Accuracy: {accuracy:.4f}, Error Rate: {error_rate:.4f}")

# 각 클래스별 정확도 계산 및 출력
for class_name in test_dataset.classes:
    class_accuracy = class_correct[class_name] / class_total[class_name] if class_total[class_name] > 0 else 0
    print(f"Class: {class_name}, Accuracy: {class_accuracy:.4f}, Total: {class_total[class_name]}")


Overall Accuracy: 0.9888, Error Rate: 0.0112
Class: Countstrike, Accuracy: 0.9925, Total: 399
Class: cartrider, Accuracy: 0.9869, Total: 306
Class: crazyarcade, Accuracy: 0.9894, Total: 282
Class: cyperse, Accuracy: 0.9974, Total: 391
Class: dunfa, Accuracy: 0.9915, Total: 352
Class: elsword, Accuracy: 0.9851, Total: 336
Class: fconline, Accuracy: 0.9792, Total: 337
Class: mabinogi, Accuracy: 0.9803, Total: 305
Class: mabinogoheros, Accuracy: 0.9985, Total: 673
Class: maplestory, Accuracy: 0.9771, Total: 306
Class: sudden, Accuracy: 0.9790, Total: 429
Class: windcountry, Accuracy: 0.9971, Total: 348
